In [1]:
%env WANDB_NOTEBOOK_NAME Train.ipynb
%env CUDA_DEVICE_ORDER PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES 1

env: WANDB_NOTEBOOK_NAME=Train.ipynb
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
from mp_20_utils import load_all_data
from tokenization import MASK_SITE
device = 'cuda'
datasets_pd, torch_datasets, site_to_ids, element_to_ids, spacegroup_to_ids, max_len = load_all_data(
    device=device, allow_retokenize=True)

In [3]:
from wyckoff_transformer import WyckoffTransformerModel, WyckoffTrainer
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 4  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 4  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability
n_space_groups = len(spacegroup_to_ids)
# Not all 230 space groups are present in the data
model = WyckoffTransformerModel(
    n_space_groups = n_space_groups,
    n_sites = len(site_to_ids),
    n_elements = len(element_to_ids),
    d_space_groups = 16,
    d_sites = 64,
    d_species = 64,
    nhead=nhead,
    d_hid=d_hid,
    nlayers=nlayers,
    dropout=dropout).to(device)

In [4]:
import torch
trainer = WyckoffTrainer(model, torch_datasets, max_len, torch.tensor(site_to_ids[MASK_SITE]).to(device))
trainer.train(epochs=2000, val_period=10)

wandb: Currently logged in as: kazeev. Use `wandb login --relogin` to force relogin


Epoch 10 val_loss_epoch 10.943184852600098 saved to checkpoints/2024-04-09_16-58-38/best_model_params.pt
Epoch 20 val_loss_epoch 6.694488525390625 saved to checkpoints/2024-04-09_16-58-38/best_model_params.pt
Epoch 30 val_loss_epoch 3.5719032287597656 saved to checkpoints/2024-04-09_16-58-38/best_model_params.pt
Epoch 40 val_loss_epoch 2.566133975982666 saved to checkpoints/2024-04-09_16-58-38/best_model_params.pt
Epoch 50 val_loss_epoch 1.73412024974823 saved to checkpoints/2024-04-09_16-58-38/best_model_params.pt
Epoch 140 val_loss_epoch 1.623763084411621 saved to checkpoints/2024-04-09_16-58-38/best_model_params.pt
Epoch 170 val_loss_epoch 1.6146345138549805 saved to checkpoints/2024-04-09_16-58-38/best_model_params.pt
Epoch 180 val_loss_epoch 1.556702733039856 saved to checkpoints/2024-04-09_16-58-38/best_model_params.pt
